In [1]:
import pandas as pd
import re
from collections import namedtuple
import pdfplumber
import os 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np

### 1. Importing and  merging pop files
### 2. Adding Metropolitans and Micropolitans

In [2]:
pop_2010_2019 = pd.read_excel('../data/raw_data/NC/pop_est_2010-2019_nc.xlsx',sheet_name='Sheet1')
pop_2010_2019 = pd.DataFrame(pop_2010_2019)
pop_2010_2019 = pop_2010_2019.drop(['Census','Estimates Base',2010,2011,2012],axis=1)
pop_2010_2019.head()

,County,2013,2014,2015,2016,2017,2018,2019
0,Alamance,154162,155384,157037,160283,163276,166514,169509
1,Alexander,37012,36987,36960,37133,37143,37331,37497
2,Alleghany,10903,10891,10856,10919,10988,11138,11137
3,Anson,25957,25965,25602,25165,24829,24470,24446
4,Ashe,26840,26764,26628,26600,26787,27099,27203


In [3]:
pop_2020_2021 = pd.read_excel('../data/raw_data/NC/pop_est_2020-2021_nc.xlsx',sheet_name='Sheet1')
pop_2020_2021 = pd.DataFrame(pop_2020_2021)
pop_2020_2021 = pop_2020_2021.drop('Estimates Base',axis=1)
pop_2020_2021.head()

,County,2020,2021
0,Alamance,171842,173877
1,Alexander,36430,36644
2,Alleghany,10899,11049
3,Anson,22030,22060
4,Ashe,26569,26711


In [4]:
pop_combined = pd.merge(pop_2010_2019,pop_2020_2021,how="left",on="County")
pop_combined['County'] = pop_combined['County'].str.strip()
pop_combined['County'] = pop_combined['County'].str.upper()
pop_combined.head()

,County,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ALAMANCE,154162,155384,157037,160283,163276,166514,169509,171842,173877
1,ALEXANDER,37012,36987,36960,37133,37143,37331,37497,36430,36644
2,ALLEGHANY,10903,10891,10856,10919,10988,11138,11137,10899,11049
3,ANSON,25957,25965,25602,25165,24829,24470,24446,22030,22060
4,ASHE,26840,26764,26628,26600,26787,27099,27203,26569,26711


In [5]:
pop_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   County  100 non-null    object
 1   2013    100 non-null    int64 
 2   2014    100 non-null    int64 
 3   2015    100 non-null    int64 
 4   2016    100 non-null    int64 
 5   2017    100 non-null    int64 
 6   2018    100 non-null    int64 
 7   2019    100 non-null    int64 
 8   2020    100 non-null    int64 
 9   2021    100 non-null    int64 
dtypes: int64(9), object(1)
memory usage: 8.6+ KB


In [6]:
metros = pd.read_excel('../data/metro_areas.xlsx',sheet_name='Filtered_States')
metros_df = pd.DataFrame(metros)
metros_nc = metros_df[metros_df['State Name']=='North Carolina']
metros_nc['County'] = metros_nc['County'].str.strip()
metros_nc['County'] = metros_nc['County'].str.upper()
metros_nc.head()

C:\Users\iulia\AppData\Local\Temp/ipykernel_35100/2973371930.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metros_nc['County'] = metros_nc['County'].str.strip()
C:\Users\iulia\AppData\Local\Temp/ipykernel_35100/2973371930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metros_nc['County'] = metros_nc['County'].str.upper()


,CBSA Title,Metropolitan/Micropolitan Statistical Area,County,State Name
1,"Burlington, NC",Metropolitan Statistical Area,ALAMANCE,North Carolina
2,"Hickory-Lenoir-Morganton, NC",Metropolitan Statistical Area,ALEXANDER,North Carolina
7,"Charlotte-Concord-Gastonia, NC-SC",Metropolitan Statistical Area,ANSON,North Carolina
21,"Washington, NC",Micropolitan Statistical Area,BEAUFORT,North Carolina
38,"Myrtle Beach-Conway-North Myrtle Beach, SC-NC",Metropolitan Statistical Area,BRUNSWICK,North Carolina


### Extracting data for years 2013 thru 2016 and combining with pop along with Metropolitans and Micropolitans

In [7]:
file_2013_2016 ='../data/raw_data/NC/NC_2012_2016.pdf'

In [8]:
pdf = pdfplumber.open(file_2013_2016)
txt = pdf.pages[42].extract_text()
txt = txt.split("\n")
new_list = txt[0:7]
new_list

['Alamance County',
 '2012 2013 2014 2015 2016 5 Year Avg.',
 'Reportable Crashes Injuries Crashes Injuries Crashes Injuries Crashes Injuries Crashes Injuries Crashes Injuries',
 'Fatal 13 13 21 23 9 9 23 23 14 15 16 17',
 'Non Fatal Injury 1,092 1,669 1,136 1,731 1,152 1,767 1,244 1,940 1,275 2,027 1,180 1,827',
 'PDO 2,157 2,349 2,574 2,558 2,798 2,487',
 'Total 3,262 1,682 3,506 1,754 3,735 1,776 3,825 1,963 4,087 2,042 3,683 1,843']

In [9]:
county = new_list[0]
county = county.replace("County","")
county = county.strip()
fatalaties = new_list[3].split(" ")
del fatalaties[0:3]
del fatalaties[-2:]
fatalaties = list(np.int_(fatalaties))
total = new_list[6].replace(",","")
total = total.split(" ")
del total[0:3]
del total[-2:]
total = list(np.int_(total))
print(county,fatalaties,total)    

Alamance [21, 23, 9, 9, 23, 23, 14, 15] [3506, 1754, 3735, 1776, 3825, 1963, 4087, 2042]


In [10]:
my_dic_2013 = {}
my_dic_2013['County'] = county
my_dic_2013['fatalaties'] = fatalaties[1]
my_dic_2013['injuries'] = total[1]
my_dic_2013['total_crashes'] = total[0]
print(my_dic_2013) 

{'County': 'Alamance', 'fatalaties': 23, 'injuries': 1754, 'total_crashes': 3506}


In [11]:
my_dic_2014 = {}
my_dic_2014['County'] = county
my_dic_2014['fatalaties'] = fatalaties[3]
my_dic_2014['injuries'] = total[3]
my_dic_2014['total_crashes'] = total[2]
my_dic_2014

{'County': 'Alamance',
 'fatalaties': 9,
 'injuries': 1776,
 'total_crashes': 3735}

In [12]:
my_dic_2015 = {}
my_dic_2015['County'] = county
my_dic_2015['fatalaties'] = fatalaties[5]
my_dic_2015['injuries'] = total[5]
my_dic_2015['total_crashes'] = total[4]
my_dic_2015

{'County': 'Alamance',
 'fatalaties': 23,
 'injuries': 1963,
 'total_crashes': 3825}

In [13]:
my_dic_2016 = {}
my_dic_2016['County'] = county
my_dic_2016['fatalaties'] = fatalaties[7]
my_dic_2016['injuries'] = total[7]
my_dic_2016['total_crashes'] = total[6]
print(my_dic_2016)

{'County': 'Alamance', 'fatalaties': 15, 'injuries': 2042, 'total_crashes': 4087}


In [14]:
fl_l_2013 = []
fl_l_2014 = []
fl_l_2015 = []
fl_l_2016 = []
pages = list(range(42,142))
pdf = pdfplumber.open(file_2013_2016)
for page in pages:
    my_dic_2013 = {}
    my_dic_2014 = {}
    my_dic_2015 = {}
    my_dic_2016 = {}
    txt =pdf.pages[page].extract_text()
    txt = txt.split("\n")
    new_list = txt[0:7]
    county = new_list[0]
    county = county.replace("County","")
    county = county.strip()
    fatalaties = new_list[3].split(" ")
    del fatalaties[0:3]
    del fatalaties[-2:]
    fatalaties = list(np.int_(fatalaties))
    total = new_list[6].replace(",","")
    total = total.split(" ")
    del total[0:3]
    del total[-2:]
    total = list(np.int_(total))
    my_dic_2013['County'] = county
    my_dic_2013['fatalaties'] = fatalaties[1]
    my_dic_2013['injuries'] = total[1]
    my_dic_2013['total_crashes'] = total[0]
    fl_l_2013.append(my_dic_2013)
    my_dic_2014['County'] = county
    my_dic_2014['fatalaties'] = fatalaties[3]
    my_dic_2014['injuries'] = total[3]
    my_dic_2014['total_crashes'] = total[2]
    fl_l_2014.append(my_dic_2014)
    my_dic_2015['County'] = county
    my_dic_2015['fatalaties'] = fatalaties[5]
    my_dic_2015['injuries'] = total[5]
    my_dic_2015['total_crashes'] = total[4]
    fl_l_2015.append(my_dic_2015)
    my_dic_2016['County'] = county
    my_dic_2016['fatalaties'] = fatalaties[7]
    my_dic_2016['injuries'] = total[7]
    my_dic_2016['total_crashes'] = total[6]
    fl_l_2016.append(my_dic_2016)
nc_df_2013 = pd.DataFrame(fl_l_2013)
nc_df_2014 = pd.DataFrame(fl_l_2014)
nc_df_2015 = pd.DataFrame(fl_l_2015) 
nc_df_2016 = pd.DataFrame(fl_l_2016)
nc_df_2013 ['County'] = nc_df_2013 ['County'].str.strip()
nc_df_2013 ['County'] = nc_df_2013 ['County'].str.upper()
nc_df_2014 ['County'] = nc_df_2014 ['County'].str.strip()
nc_df_2014 ['County'] = nc_df_2014 ['County'].str.upper()
nc_df_2015 ['County'] = nc_df_2015 ['County'].str.strip()
nc_df_2015 ['County'] = nc_df_2015 ['County'].str.upper()
nc_df_2016 ['County'] = nc_df_2016 ['County'].str.strip()
nc_df_2016 ['County'] = nc_df_2016 ['County'].str.upper()
df_pop_nc13 = pd.merge(nc_df_2013,pop_combined[['County', 2013]],on="County",how='left')
df_pop_nc14 = pd.merge(nc_df_2014,pop_combined[['County', 2014]],on="County",how='left')
df_pop_nc15 = pd.merge(nc_df_2015,pop_combined[['County', 2015]],on="County",how='left')
df_pop_nc16 = pd.merge(nc_df_2016,pop_combined[['County', 2016]],on="County",how='left')
df_comb_nc13 = pd.merge(df_pop_nc13,metros_nc[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_comb_nc14 = pd.merge(df_pop_nc14,metros_nc[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_comb_nc15 = pd.merge(df_pop_nc15,metros_nc[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_comb_nc16 = pd.merge(df_pop_nc16,metros_nc[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_comb_nc13.rename(columns = {2013:'population'}, inplace = True)
df_comb_nc13.insert(0,'Year',2013)
df_comb_nc14.rename(columns = {2014:'population'}, inplace = True)
df_comb_nc14.insert(0,'Year',2014)
df_comb_nc15.rename(columns = {2015:'population'}, inplace = True)
df_comb_nc15.insert(0,'Year',2015)
df_comb_nc16.rename(columns = {2016:'population'}, inplace = True)
df_comb_nc16.insert(0,'Year',2016)
df_comb_nc13.to_excel('../data/cleaned_data/2013_NC.xlsx', engine='xlsxwriter',index=False)
df_comb_nc14.to_excel('../data/cleaned_data/2014_NC.xlsx', engine='xlsxwriter',index=False)
df_comb_nc15.to_excel('../data/cleaned_data/2015_NC.xlsx', engine='xlsxwriter',index=False)
df_comb_nc16.to_excel('../data/cleaned_data/2016_NC.xlsx', engine='xlsxwriter',index=False)

### Extracting data for years 2017 thru 2020 and combining with pop along with Metropolitans and Micropolitans

In [15]:
file_2017_2020 ='../data/raw_data/NC/NC_2016_2020.pdf'

In [16]:
pdf = pdfplumber.open(file_2017_2020)
txt = pdf.pages[42].extract_text()
txt = txt.split("\n")
new_list = txt[0:7]
new_list

['Alamance County',
 '2016 2017 2018 2019 2020 5 Year Avg.',
 'Reportable Crashes Injuries Crashes Injuries Crashes Injuries Crashes Injuries Crashes Injuries Crashes Injuries',
 'Fatal 14 15 22 23 17 19 14 15 29 29 19 20',
 'Non Fatal Injury 1,275 2,027 1,247 1,931 1,203 1,833 1,270 1,987 1,131 1,774 1,225 1,910',
 'PDO 2,798 2,761 3,105 2,962 2,810 2,887',
 'Total 4,087 2,042 4,030 1,954 4,325 1,852 4,246 2,002 3,970 1,803 4,132 1,931']

In [17]:
county = new_list[0]
county = county.replace("County","")
county = county.strip()
fatalaties = new_list[3].split(" ")
del fatalaties[0:3]
del fatalaties[-2:]
total = new_list[6].split(" ")
del total[0:3]
del total[-2:]
print(county,fatalaties,total)

Alamance ['22', '23', '17', '19', '14', '15', '29', '29'] ['4,030', '1,954', '4,325', '1,852', '4,246', '2,002', '3,970', '1,803']


In [18]:
fl_l_2017 = []
fl_l_2018 = []
fl_l_2019 = []
fl_l_2020 = []
pages = list(range(42,142))
pdf = pdfplumber.open(file_2017_2020)
for page in pages:
    my_dic_2017 = {}
    my_dic_2018 = {}
    my_dic_2019 = {}
    my_dic_2020 = {}
    txt =pdf.pages[page].extract_text()
    txt = txt.split("\n")
    new_list = txt[0:7]
    county = new_list[0]
    county = county.replace("County","")
    county = county.strip()
    fatalaties = new_list[3].split(" ")    
    del fatalaties[0:3]
    del fatalaties[-2:]
    fatalaties = list(np.int_(fatalaties))
    total = new_list[6].replace(",","")
    total = total.split(" ")
    del total[0:3]
    del total[-2:]
    total = list(np.int_(total))
    my_dic_2017['County'] = county
    my_dic_2017['fatalaties'] = fatalaties[1]
    my_dic_2017['injuries'] = total[1]
    my_dic_2017['total_crashes'] = total[0]
    fl_l_2017.append(my_dic_2017)
    my_dic_2018['County'] = county
    my_dic_2018['fatalaties'] = fatalaties[3]
    my_dic_2018['injuries'] = total[3]
    my_dic_2018['total_crashes'] = total[2]
    fl_l_2018.append(my_dic_2018)
    my_dic_2019['County'] = county
    my_dic_2019['fatalaties'] = fatalaties[5]
    my_dic_2019['injuries'] = total[5]
    my_dic_2019['total_crashes'] = total[4]
    fl_l_2019.append(my_dic_2019)
    my_dic_2020['County'] = county
    my_dic_2020['fatalaties'] = fatalaties[7]
    my_dic_2020['injuries'] = total[7]
    my_dic_2020['total_crashes'] = total[6]
    fl_l_2020.append(my_dic_2020)
nc_df_2017 = pd.DataFrame(fl_l_2017)
nc_df_2018 = pd.DataFrame(fl_l_2018)
nc_df_2019 = pd.DataFrame(fl_l_2019) 
nc_df_2020 = pd.DataFrame(fl_l_2020)
nc_df_2017 ['County'] = nc_df_2017 ['County'].str.strip()
nc_df_2017 ['County'] = nc_df_2017 ['County'].str.upper()
nc_df_2018 ['County'] = nc_df_2018 ['County'].str.strip()
nc_df_2018 ['County'] = nc_df_2018 ['County'].str.upper()
nc_df_2019 ['County'] = nc_df_2019 ['County'].str.strip()
nc_df_2019 ['County'] = nc_df_2019 ['County'].str.upper()
nc_df_2020 ['County'] = nc_df_2020 ['County'].str.strip()
nc_df_2020 ['County'] = nc_df_2020 ['County'].str.upper()
df_pop_nc17 = pd.merge(nc_df_2017,pop_combined[['County', 2017]],on="County",how='left')
df_pop_nc18 = pd.merge(nc_df_2018,pop_combined[['County', 2018]],on="County",how='left')
df_pop_nc19 = pd.merge(nc_df_2019,pop_combined[['County', 2019]],on="County",how='left')
df_pop_nc20 = pd.merge(nc_df_2020,pop_combined[['County', 2020]],on="County",how='left')
df_comb_nc17 = pd.merge(df_pop_nc17,metros_nc[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_comb_nc18 = pd.merge(df_pop_nc18,metros_nc[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_comb_nc19 = pd.merge(df_pop_nc19,metros_nc[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_comb_nc20 = pd.merge(df_pop_nc20,metros_nc[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
df_comb_nc17.rename(columns = {2017:'population'}, inplace = True)
df_comb_nc17.insert(0,'Year',2017)
df_comb_nc18.rename(columns = {2018:'population'}, inplace = True)
df_comb_nc18.insert(0,'Year',2018)
df_comb_nc19.rename(columns = {2019:'population'}, inplace = True)
df_comb_nc19.insert(0,'Year',2019)
df_comb_nc20.rename(columns = {2020:'population'}, inplace = True)
df_comb_nc20.insert(0,'Year',2020)
df_comb_nc17.to_excel('../data/cleaned_data/2017_NC.xlsx', engine='xlsxwriter',index=False)
df_comb_nc18.to_excel('../data/cleaned_data/2018_NC.xlsx', engine='xlsxwriter',index=False)
df_comb_nc19.to_excel('../data/cleaned_data/2019_NC.xlsx', engine='xlsxwriter',index=False)
df_comb_nc20.to_excel('../data/cleaned_data/2020_NC.xlsx', engine='xlsxwriter',index=False)

### Combining all the files into a master file

In [19]:
data_file_folder ='../data/cleaned_data/'
df = []
for file in os.listdir(data_file_folder):
        if file.endswith('_NC.xlsx'):             
             df.append(pd.read_excel(os.path.join(data_file_folder,file),sheet_name='Sheet1'))
df_nc_master = pd.concat(df, axis = 0)

In [20]:
df_nc_master.isna().sum()

Year                                            0
County                                          0
fatalaties                                      0
injuries                                        0
total_crashes                                   0
population                                      0
CBSA Title                                    200
Metropolitan/Micropolitan Statistical Area    200
dtype: int64

In [21]:
df_nc_master.head(10)

,Year,County,fatalaties,injuries,total_crashes,population,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,2013,ALAMANCE,23,1754,3506,154162,"Burlington, NC",Metropolitan Statistical Area
1,2013,ALEXANDER,8,212,455,37012,"Hickory-Lenoir-Morganton, NC",Metropolitan Statistical Area
2,2013,ALLEGHANY,3,98,184,10903,NaN,NaN
3,2013,ANSON,7,370,597,25957,"Charlotte-Concord-Gastonia, NC-SC",Metropolitan Statistical Area
4,2013,ASHE,4,206,431,26840,NaN,NaN
5,2013,AVERY,2,151,289,17654,NaN,NaN
6,2013,BEAUFORT,13,435,961,47382,"Washington, NC",Micropolitan Statistical Area
7,2013,BERTIE,5,251,431,20326,NaN,NaN
8,2013,BLADEN,16,499,807,34618,NaN,NaN
9,2013,BRUNSWICK,13,904,2111,114922,"Myrtle Beach-Conway-North Myrtle Beach, SC-NC",Metropolitan Statistical Area


In [22]:
df_nc_master.tail(10)

,Year,County,fatalaties,injuries,total_crashes,population,CBSA Title,Metropolitan/Micropolitan Statistical Area
90,2020,VANCE,15,611,1345,42529,"Henderson, NC",Micropolitan Statistical Area
91,2020,WAKE,88,10130,25546,1133553,"Raleigh-Cary, NC",Metropolitan Statistical Area
92,2020,WARREN,10,179,374,18598,NaN,NaN
93,2020,WASHINGTON,0,63,238,10955,NaN,NaN
94,2020,WATAUGA,7,341,1181,54072,"Boone, NC",Micropolitan Statistical Area
95,2020,WAYNE,30,1272,3130,117226,"Goldsboro, NC",Metropolitan Statistical Area
96,2020,WILKES,13,630,1300,65871,"North Wilkesboro, NC",Micropolitan Statistical Area
97,2020,WILSON,19,1170,2680,78811,"Wilson, NC",Micropolitan Statistical Area
98,2020,YADKIN,12,266,683,37182,"Winston-Salem, NC",Metropolitan Statistical Area
99,2020,YANCEY,2,98,201,18475,NaN,NaN


In [23]:
df_nc_master.to_excel('../data/NC.xlsx', engine='xlsxwriter',index=False)

In [24]:
df_nc_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800 entries, 0 to 99
Data columns (total 8 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Year                                        800 non-null    int64 
 1   County                                      800 non-null    object
 2   fatalaties                                  800 non-null    int64 
 3   injuries                                    800 non-null    int64 
 4   total_crashes                               800 non-null    int64 
 5   population                                  800 non-null    int64 
 6   CBSA Title                                  600 non-null    object
 7   Metropolitan/Micropolitan Statistical Area  600 non-null    object
dtypes: int64(5), object(3)
memory usage: 56.2+ KB


In [25]:
df_nc_master['injuries'] = df_nc_master['injuries'].str.replace(',', '').astype(float)
df_nc_master['total_crashes'] = df_nc_master['total_crashes'].str.replace(',', '').astype(float)

AttributeError: Can only use .str accessor with string values!

In [ ]:
df_nc_master_grouped = df_nc_master.groupby(['Year','CBSA Title','Metropolitan/Micropolitan Statistical Area']).sum()
df_nc_master_grouped 

In [ ]:
unstacked_df = df_nc_master_grouped.unstack()
unstacked_df